# CDAT Migration Regression Testing Notebook (`.png` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.


## Setup Code


In [8]:
from collections import defaultdict
import glob
from typing import Tuple

import numpy as np
import xarray as xr
from e3sm_diags.derivations.derivations import DERIVED_VARIABLES
from auxiliary_tools.cdat_regression_testing.utils import get_image_diffs


# TODO: Update SET_NAME and SET_DIR
SET_NAME = "diurnal_cycle"
SET_DIR = "666-diurnal-cycle"

DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{SET_DIR}/{SET_NAME}/**"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.png"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/{SET_NAME}/**"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.png"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [9]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_count = 0

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(SET_DIR, "main")

        if fp_dev not in MAIN_GLOB:
            print(f"No production file found to compare with {fp_dev}!")
            missing_count += 1

    for fp_dev in DEV_GLOB:
        fp_main = fp_main.replace("main", SET_DIR)

        if fp_main not in DEV_GLOB:
            print(f"No development file found to compare with {fp_main}!")
            missing_count += 1

    print(f"Number of files missing: {missing_count}")

## 1. Check for matching and equal number of files


In [10]:
_check_if_files_found()

In [11]:
_check_if_missing_files()

Number of files missing: 0


In [12]:
_check_if_matching_filecount()

Matching file count (30 and 30).


## 2 Compare the plots between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [15]:
for main_path, dev_path in zip(MAIN_GLOB, DEV_GLOB):
    print("Comparing:")
    print(f"    * {main_path}")
    print(f"    * {dev_path}")

    try:
        get_image_diffs(dev_path, main_path)
    except TypeError as e:
        if "cannot unpack non-iterable NoneType object" in str(e):
            print("     * Plots are identical")
            continue

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-20S20N.png
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/666-diurnal-cycle/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-20S20N.png
/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-20S20N.png
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-50S50N.png
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/666-diurnal-cycle/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-50S50N.png
/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-50S50N.png
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-Amazon.png
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/666-diurnal

### Results

TODO: FILL THIS IN
